In [ ]:
import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import get_test_data
from mpl_toolkits.mplot3d import Axes3D
import math
from math import pi


In [ ]:
def normaldefinition_3D_real(void_data, k):
    dist = distance.squareform(distance.pdist(void_data))
    closest = np.argsort(dist, axis=1)
    
    total_pts = np.size(closest,0)
    planes = np.zeros((total_pts, 6))
    
    fig1=plt.figure()
    ax = fig1.add_subplot(1,1,1, projection='3d')
    ax.scatter(x,y,z, facecolors='none',edgecolors='r')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    for i in range(total_pts):
        normal_vect, xmn, ymn, zmn, knn_pt_coord = tangentplane_3D_real(closest[i,:], void_data,k)
        
        planes[i,0:3] = normmal_vect
        planes[i,3:6] = np.array([xmn, ymn, zmn])
        
        ax.scatter(xmn,ymn,zmn, facecolors='none',edgecolors = 'c')
        ax.plot([xmn,xmn+normalvect[0]],[ymn,ymn+normalvect[1]],[zmn,zmn+normalvect[2]])
        plt.show()
        plt.hold(True)
    planes_consist = normalconsistency_3D_real(planes)
    return  planes,planes_consist


In [ ]:
def tangentplane_3D_real(closest_pt,ellipsoid_data,k):
    knn_pt_id = closest_pt[0:k] # Retain only the indexes of the k-closest points
    nb_points = np.size(knn_pt_id)
    knn_pt_coord = np.zeros((nb_points,3)) 
    
    for i in range(nb_points):
        point_i = knn_pt_id[i]
        
        knn_pt_coord[i,:] = ellipsoid_data[point_i,:]
    
    #"Principal component analysis (PCA)"
    
    #"Centorid calculation"
        
    xmn = np.mean(knn_pt_coord[:,0])
    ymn = np.mean(knn_pt_coord[:,1])
    zmn = np.mean(knn_pt_coord[:,2])
    
    c=np.zeros((np.size(knn_pt_coord,0),3))
    
    c[:,0] = knn_pt_coord[:,0]-xmn
    c[:,1] = knn_pt_coord[:,1]-ymn
    c[:,2] = knn_pt_coord[:,2]-zmn
    
    #"Covariance matrix"
    
    cov=np.zeros((3,3))    
    
    cov[0,0] = np.dot(c[:,0],c[:,0])
    cov[0,1] = np.dot(c[:,0],c[:,1])
    cov[0,2] = np.dot(c[:,0],c[:,2])
    
    cov[1,0] = cov[0,1]
    cov[1,1] = np.dot(c[:,1],c[:,1])
    cov[1,2] = np.dot(c[:,1],c[:,2])
    
    cov[2,0] = cov[0,2]
    cov[2,1] = cov[1,2]
    cov[2,2] = np.dot(c[:,2],c[:,2])
   
    #"Single value decomposition (SVD)"
    
    u,s,vh = np.linalg.svd(cov) # U contains the orthonormal eigenvectors and S contains the eigenvectors

    #"Selection of minimum eigenvalue"
    
    minevindex = np.argmin(s)
    
    #"Selection of orthogonal vector corresponing to this eigenvalue --> vector normal to the plane defined by the kpoints"
    
    normal_vect = u[:,minevindex]

    return normal_vect, xmn, ymn,zmn,knn_pt_coord


In [ ]:
def normalconsistency_3D_real(planes):
    
    nbnormals = np.size(planes, 0)
    planes_consist=np.zeros((nbnormals,6))
    planes_consist[:, 3:6] = planes[:, 3:6] # We just copy the columns corresponding to the coordinates of the centroids (from 3th to 5th)
    
    #""" Try the atan2 function : https://uk.mathworks.com/help/vision/ref/pcnormals.html#buxdmoj"""
    
    sensorcentre=np.array([0,0,0])
    
    for i in range(nbnormals):
        p1 = (sensorcentre - planes[i,3:6]) / np.linalg.norm(sensorcentre - planes[i,3:6]) # Vector from the centroid to the centre of the ellipsoid (here the sensor is placed)
        p2 = planes[i,0:3]
        
        angle = math.atan2(np.linalg.norm(np.cross(p1,p2)), np.dot(p1,p2) ) # Angle between the centroid-sensor and plane normal
       
        
        if (angle >= -pi/2 and angle <= pi/2): # (angle >= -pi/2 and angle <= pi/2):
            
            planes_consist[i,0] = -planes[i,0]
            planes_consist[i,1] = -planes[i,1]
            planes_consist[i,2] = -planes[i,2]  
            
        else:
            
            planes_consist[i,0] = planes[i,0]
            planes_consist[i,1] = planes[i,1]
            planes_consist[i,2] = planes[i,2]

    return planes_consist


In [ ]:
filepath = '/Users/qwert/Documents/TASKAR/ply_data/west3_subsample.ply'

In [ ]:
points  = np.fromfile(filepath,dtype=float)
points

In [ ]:
type(points)

In [2]:
!pip install open3d

In [7]:
import open3d as o3d

ImportError: dlopen(/Applications/anaconda3/lib/python3.8/site-packages/open3d/cpu/pybind.cpython-38-darwin.so, 0x0002): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <3DD86FDF-C162-3CA9-BE4C-3A6B0DCEBF39> /Applications/anaconda3/lib/python3.8/site-packages/open3d/cpu/pybind.cpython-38-darwin.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [6]:
! pip install open3d